In [3]:
import requests
import os
from dotenv import load_dotenv
import cv2
import numpy as np
import wave
import contextlib
from moviepy import VideoFileClip, AudioFileClip

load_dotenv('.env')

ModuleNotFoundError: No module named 'moviepy'

In [2]:
heygen_apikey = os.environ['HEYGEN_APIKEY']
elevenlabs_apikey = os.environ['ELEVENLABS_APIKEY']

In [3]:
def get_audio_duration(path):
    with contextlib.closing(wave.open(path,'r')) as f:
        return f.getnframes() / float(f.getframerate())

def create_talking_head_video(image_path, audio_path, output_path="talking_head.mp4", fps=25):
    img = cv2.imread(image_path)
    height, width, _ = img.shape

    duration = get_audio_duration(audio_path)
    frame_count = int(duration * fps)

    mouth_open = False
    frames = []

    for i in range(frame_count):
        frame = img.copy()

        # Simulate mouth movement by drawing an ellipse (very basic)
        if i % 5 == 0:
            mouth_open = not mouth_open

        if mouth_open:
            cv2.ellipse(frame, (width//2, int(height*0.75)), (50, 25), 0, 0, 360, (0,0,0), -1)

        frames.append(frame)

    # Write frames to video
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
    for frame in frames:
        out.write(frame)
    out.release()


In [ ]:
def mux_audio_video(video_path, audio_path, output_path="final_talking_head.mp4"):
    video_clip = VideoFileClip(video_path)
    audio_clip = AudioFileClip(audio_path)

    final_clip = video_clip.set_audio(audio_clip)
    final_clip.write_videofile(output_path, codec="libx264", audio_codec="aac")


ModuleNotFoundError: No module named 'moviepy.editor'

In [4]:
url = 'https://api.heygen.com/v2/voices'

headers = {
    'Accept': 'application/json',
    'X-Api-Key': heygen_apikey
}

In [5]:
resp = requests.get(url=url, headers=headers)

In [9]:
video_generate_url = 'https://api.heygen.com/v2/video/generate'

In [8]:
data = {
    "video_inputs": [
        {
            "character": {
                "type": "avatar",
                "avatar_id": "Lina_Dress_Sitting_Side_public",
                "avatar_style": "normal"
            },
            "voice": {
                "type": "text",
                "input_text": "Welcome to the HeyGen API!",
                "voice_id": "119caed25533477ba63822d5d1552d25",
                "speed": 1.1
            }
        }
    ],
    "dimension": {
        "width": 1280,
        "height": 720
    }
}

In [10]:
video_resp = requests.post(url=video_generate_url, headers=headers, json=data)

In [21]:
if 'data' in video_resp.json():
    video_id = video_resp.json()['data']['video_id']

In [22]:
video_id

'4ba7febe88da47228a45ce8c1e11ad3f'

In [23]:
generated_url = f"https://api.heygen.com/v1/video_status.get?video_id={video_id}"

In [26]:
resp = requests.get(url=generated_url, headers=headers)

In [27]:
resp.json()

{'code': 100,
 'data': {'callback_id': None,
  'caption_url': None,
  'created_at': 1749871145,
  'duration': None,
  'error': {'code': 'AVATAR_USAGE_NOT_PERMITTED',
   'detail': 'Please upgrade your plan to use this avatar: None',
   'message': 'Please upgrade your plan to use this avatar: None'},
  'gif_url': None,
  'id': '4ba7febe88da47228a45ce8c1e11ad3f',
  'status': 'failed',
  'thumbnail_url': None,
  'video_url': None,
  'video_url_caption': None},
 'message': 'Success'}